In [ ]:
import socket
import numpy as np
import cv2 as cv
import threading

print('-------------------------------Client-----------------------------')

# client program socket to connect to the server program
s = socket.socket()
s.bind(("", 6666))  # empty means local system
server_ip = input("enter server ip: ")
server_port = input("enter server port number: ")

s.connect((server_ip, int(server_port))) 
s.send(b"connected")  # sending string as data
camera = cv.VideoCapture(0) # starting the camera i.e. laptop webcam


# Client works as a receiver
def receiver():
    print("Entered into live stream with Server")
    while True:
        data = s.recv(100000000)  # receiving data with the size limit
        if(data == b'finished'): # to stop receiving and stop camera
            print("Finished")
            camera.release()
            s.close()
        else:  # converting the byte data into numpy array
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3: # changing the array shape and getting the video
                cv.imshow('From Server', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13: # camera closing condition
                    s.send(b'finished')
                    camera.release()
                    cv.destroyAllWindows()
                    break
            else:
                print('Frames are getting lost')

# Client works as a sender             
def sender():
    while True: 
        status, photo = camera.read()
        photo = cv.resize(photo, (640, 480))
        if status:
            s.send(np.ndarray.tobytes(photo)) #converting numpy array into bytes so that we can send data
        else: print("Image not found")
    camera.release()

# threads to run both the functions
threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()